# Complete Enumeration of N=6 Two-Layer Networks

**Purpose**: Enumerate all non-isomorphic two-layer multiplex configurations on N=6 nodes  
**Output**: CSV with θ,φ coefficients for each unique configuration  
**Language**: Julia 1.11+

---

## Overview

This notebook performs exhaustive enumeration of two-layer multiplex networks:

1. **Graph enumeration**: Use nauty's `geng` to generate all 112 non-isomorphic connected graphs on 6 vertices
2. **Rooted configurations**: For each graph, identify orbit representatives for single-mutant placement
3. **Multiplex pairs**: Enumerate all pairs of rooted graphs with all layer alignments
4. **Canonical deduplication**: Use lexicographic canonical codes to eliminate isomorphic configurations
5. **Compute θ,φ**: Calculate strategy correlation coefficients for each unique configuration

### Prerequisites

- **nauty** (`geng` binary): Install via `brew install nauty` (macOS) or `apt install nauty` (Linux)
- Julia packages: `Graphs`, `IterativeSolvers`, `SparseArrays`

---
## 1. Configuration & Imports

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

const N = 6                              # Network size (fixed for this enumeration)
const OUTPUT_FILE = "N6_enumeration.csv" # Output CSV path

# Locate nauty binaries (adjust if not on PATH)
const GENG = something(Sys.which("geng"), "/opt/homebrew/bin/geng", "/usr/bin/geng")
@assert isfile(GENG) "Could not find geng at $GENG. Install nauty first."

println("Configuration:")
println("  N = $N")
println("  Output = $OUTPUT_FILE")
println("  geng = $GENG")

In [ ]:
using Combinatorics
using Printf
using Statistics
using LinearAlgebra
using IterativeSolvers
using SparseArrays

---
## 2. Core θ,φ Solver (Shared with multilayer_egt.ipynb)

These functions compute strategy correlation coefficients for multilayer networks.

In [ ]:
# -----------------------------------------------------------------------------
# NOTE: Core functions shared with multilayer_egt.ipynb
# If refactoring, extract to MultilayerEGT.jl module
# -----------------------------------------------------------------------------

"""
    edge_seq_rearrangement(edge_seq)

Rearrange edge sequence by layer ID and starting node ID.

# Arguments
- `edge_seq::Array{Int64,2}`: Edge list [node_i, node_j, weight, layer_id]

# Returns
- Rearranged edge sequence matrix
"""
function edge_seq_rearrangement(edge_seq::Array{Int64,2})
    layer_list = setdiff(edge_seq[:,4], [])
    L = maximum(layer_list)
    N = maximum(edge_seq[:,1])
    
    edge_seq_rearranged = zeros(Int64, size(edge_seq))
    id = 1
    for ell in layer_list
        edge_list_ell = findall(isequal(ell), edge_seq[:,4])
        M = sparse(edge_seq[edge_list_ell,1], edge_seq[edge_list_ell,2], edge_seq[edge_list_ell,3])
        inds = findall(!iszero, M)
        a, b = getindex.(inds, 1), getindex.(inds, 2)
        len = length(a)
        edge_seq_rearranged[id:id+len-1, :] = hcat(b, a, M[inds], fill(ell, len))
        id += len
    end
    return edge_seq_rearranged
end

"""
    beta_gamma(edge_seq, xi_seq)

Solve linear systems for β_i, β_ij, γ_ij coefficients.

# Arguments
- `edge_seq::Array{Int64,2}`: Edge sequence [node_i, node_j, weight, layer_id]
- `xi_seq::Array{Int64,2}`: Strategy config [node_id, strategy, layer_id]

# Returns
- `solution::SparseMatrix`: Column 1 = β_ij, columns 2+ = γ_ij for each layer pair
"""
function beta_gamma(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2})
    N = maximum(edge_seq[:,1])
    L = maximum(edge_seq[:,4])
    
    # Build transition matrix
    M = spzeros(Float64, N*L, N)
    for ell = 1:L
        edge_list_ell = findall(isequal(ell), edge_seq[:,4])
        M[(ell-1)*N+1:ell*N, :] = sparse(edge_seq[edge_list_ell,1], edge_seq[edge_list_ell,2], 
                                          edge_seq[edge_list_ell,3], N, N)
    end
    
    # Stationary distribution
    pi = spzeros(Float64, L*N)
    for ell = 1:L
        pi_2 = sum(M[(ell-1)*N+1:ell*N, :], dims=2)[:,1]
        presence = findall(!iszero, pi_2)
        pi_transient = spzeros(Float64, N)
        pi_transient[presence] = pi_2[presence] / sum(pi_2)
        pi[(ell-1)*N+1:ell*N] = pi_transient
        M_transient = M[(ell-1)*N+1:ell*N, :]
        M_transient[presence, :] ./= pi_2[presence]
        M[(ell-1)*N+1:ell*N, :] = M_transient
    end

    # Strategy configuration
    xi = spzeros(Float64, L*N)
    for ell = 1:L
        presence = findall(isequal(ell), xi_seq[:,3])
        xi_t = spzeros(Int64, N)
        xi_t[xi_seq[presence,1]] = xi_seq[presence,2]
        xi[(ell-1)*N+1:ell*N] = xi_t
    end

    # Solve for β_i
    presence_list = findall(!iszero, pi[1:N])
    N1 = length(presence_list)
    MatA_i = M[1:N, :] - sparse(Matrix(1.0I, N, N))
    pi_list = findall(!isequal(presence_list[1]), 1:N)
    MatA_i[:, pi_list] .-= MatA_i[:, presence_list[1]] * transpose(pi[pi_list] / pi[presence_list[1]])
    
    xi_hat = sum(pi[1:N] .* xi[1:N])
    MatB_i = (xi_hat * ones(N) - xi[1:N]) * N1
    
    beta_i = zeros(Float64, N)
    beta_i[pi_list] = idrs(MatA_i[pi_list, pi_list], MatB_i[pi_list])
    beta_i[presence_list[1]] = -sum(pi[1:N] .* beta_i) / pi[presence_list[1]]

    # Solve for β_ij (edge-level)
    Mat1 = M[1:N, :]
    inds = findall(!iszero, Mat1)
    a, b = getindex.(inds, 1), getindex.(inds, 2)
    vec1 = transpose(1:N)
    
    X1 = N * (a .- 1) * ones(Int, 1, N) + ones(Int, length(a)) * vec1
    Y1 = N * (b .- 1) * ones(Int, 1, N) + ones(Int, length(b)) * vec1
    W1 = Mat1[inds] * ones(Int, 1, N)
    vec2 = transpose((0:N-1) * N)
    X2 = a * ones(Int, 1, N) + ones(Int, length(a)) * vec2
    Y2 = b * ones(Int, 1, N) + ones(Int, length(b)) * vec2
    W2 = Mat1[inds] * ones(Int, 1, N)

    MatA_ij = sparse(vec(X1), vec(Y1), vec(W1), N^2, N^2)/2 + 
              sparse(vec(X2), vec(Y2), vec(W2), N^2, N^2)/2 - sparse(1.0I, N^2, N^2)
    
    beta_obtained = [(i-1)*N + i for i = 1:N]
    beta_missed = setdiff(1:N^2, beta_obtained)
    absence_list = findall(iszero, pi[1:N])
    
    MatB_ij = -sum(MatA_ij[:, beta_obtained] .* transpose(beta_i), dims=2)
    Mat = xi_hat * ones(N, N) * N1/2 - xi[1:N] * transpose(xi[1:N]) * N1/2
    Mat[absence_list, :] .= 0; Mat[:, absence_list] .= 0
    MatB_ij .+= vec(transpose(Mat))
    
    beta_ij = zeros(Float64, N^2)
    beta_ij[beta_missed] = idrs(MatA_ij[beta_missed, beta_missed], MatB_ij[beta_missed])
    beta_ij[beta_obtained] = beta_i

    solution = spzeros(Float64, N^2, L)
    solution[:, 1] = beta_ij
    
    # Solve for γ_ij (inter-layer correlations)
    if L > 1
        for ell = 2:L
            xi_ell = xi[(ell-1)*N+1:ell*N]
            pi_ell = pi[(ell-1)*N+1:ell*N]
            presence_ell = findall(!iszero, pi_ell)
            N_ell = length(presence_ell)
            
            GammaA_ij = sparse(vec(X1), vec(Y1), vec(W1), N^2, N^2) * (N_ell-1)/(N1+N_ell-1)
            Mat_ell = M[(ell-1)*N+1:ell*N, :]
            inds_ell = findall(!iszero, Mat_ell)
            a_ell, b_ell = getindex.(inds_ell, 1), getindex.(inds_ell, 2)
            X_ell = a_ell * ones(Int, 1, N) + ones(Int, length(a_ell)) * vec2
            Y_ell = b_ell * ones(Int, 1, N) + ones(Int, length(b_ell)) * vec2
            W_ell = Mat_ell[inds_ell] * ones(Int, 1, N)
            GammaA_ij += sparse(vec(X_ell), vec(Y_ell), vec(W_ell), N^2, N^2) * (N1-1)/(N1+N_ell-1)
            
            vec_g1 = vec(ones(Int, N) * vec1)
            vec_g2 = vec(transpose(vec1) * ones(Int, 1, N))
            X_g1 = N*(a .- 1)*ones(Int, 1, N^2) + ones(Int, length(a))*transpose(vec_g1)
            Y_g1 = N*(b .- 1)*ones(Int, 1, N^2) + ones(Int, length(b))*transpose(vec_g2)
            W_g1 = Mat1[inds] * ones(Int, 1, N^2)
            vec_g3 = (vec_g1 .- 1) * N
            vec_g4 = (vec_g2 .- 1) * N
            X_g2 = a_ell*ones(Int, 1, N^2) + ones(Int, length(a_ell))*transpose(vec_g3)
            Y_g2 = b_ell*ones(Int, 1, N^2) + ones(Int, length(b_ell))*transpose(vec_g4)
            W_g2 = Mat_ell[inds_ell] * ones(Int, 1, N^2)
            GammaA_ij += sparse(vec(X_g1), vec(Y_g1), vec(W_g1), N^2, N^2) .*
                         sparse(vec(X_g2), vec(Y_g2), vec(W_g2), N^2, N^2) / (N1+N_ell-1)
            GammaA_ij -= sparse(1.0I, N^2, N^2)
            
            pi_t = pi[1:N] .* pi_ell
            pres_g = findall(!iszero, pi_t)
            del = (pres_g .- 1) * N + pres_g
            GammaA_ij[:, del] .-= GammaA_ij[:, del[1]] * transpose(pi[pres_g]) / pi[pres_g[1]]
            pi_list_g = findall(!isequal(del[1]), 1:N^2)
            
            xi_ell_hat = sum(pi_ell .* xi_ell)
            GammaB = (xi_hat * xi_ell_hat * ones(N, N) - xi[1:N] * transpose(xi_ell)) * N1*N_ell/(N1+N_ell-1)
            pres1 = findall(iszero, pi[1:N])
            pres_e = findall(iszero, pi_ell)
            GammaB[pres1, :] .= 0; GammaB[:, pres_e] .= 0
            GammaB = vec(transpose(GammaB))
            
            gamma = zeros(Float64, N^2)
            gamma[pi_list_g] = idrs(GammaA_ij[pi_list_g, pi_list_g], GammaB[pi_list_g])
            gamma[del[1]] = -sum(pi[pres_g] .* gamma[del]) / pi[pres_g[1]]
            solution[:, ell] = gamma
        end
    end
    return solution
end

"""
    bc_multilayer_DB(edge_seq, xi_seq)

Compute θ,φ coefficients for death-Birth dynamics.

# Returns
- `theta_phi::Matrix{Float64}`: 4×L matrix where column 1 = [θ₀, θ₁, θ₂, θ₃], 
  columns 2+ = [φ₀₀, φ₀₁, φ₂₀, φ₂₁] for inter-layer correlations
"""
function bc_multilayer_DB(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2})
    edge_seq = edge_seq_rearrangement(edge_seq)
    solution = beta_gamma(edge_seq, xi_seq)
    N, L = maximum(edge_seq[:,1]), maximum(edge_seq[:,4])
    
    M = spzeros(Float64, N*L, N)
    pi = spzeros(Float64, L*N)
    xi = spzeros(Float64, L*N)
    
    for ell = 1:L
        idx = (ell-1)*N+1:ell*N
        edge_list = findall(isequal(ell), edge_seq[:,4])
        M[idx, :] = sparse(edge_seq[edge_list,1], edge_seq[edge_list,2], edge_seq[edge_list,3], N, N)
        pi_2 = sum(M[idx, :], dims=2)[:,1]
        pres = findall(!iszero, pi_2)
        pi_t = spzeros(Float64, N); pi_t[pres] = pi_2[pres] / sum(pi_2)
        pi[idx] = pi_t
        M[idx, :][pres, :] ./= pi_2[pres]
        xi_t = spzeros(Int64, N)
        xi_pres = findall(isequal(ell), xi_seq[:,3])
        xi_t[xi_seq[xi_pres,1]] = xi_seq[xi_pres,2]
        xi[idx] = xi_t
    end

    beta = transpose(reshape(solution[:,1], N, :))
    M1 = M[1:N, :]
    theta_phi = zeros(Float64, 4, L)
    theta_phi[:,1] = [sum(pi[1:N].*xi[1:N]), sum(pi[1:N].*M1.*beta), 
                      sum(pi[1:N].*diag(beta)), sum(pi[1:N].*transpose(M1).*transpose(beta))]
    
    for ell = 2:L
        idx = (ell-1)*N+1:ell*N
        gamma = transpose(reshape(solution[:,ell], N, :))
        theta_phi[:,ell] = [sum(pi[idx].*xi[idx]), sum(pi[1:N].*transpose(M[idx,:]).*gamma.*gamma),
                            sum(pi[1:N].*diag(gamma)), sum(pi[1:N].*M[idx,:].*transpose(gamma).*gamma)]
    end
    return theta_phi
end

---
## 3. Graph Enumeration Utilities

In [ ]:
# Upper-triangular positions for N=6 adjacency encoding
const UPPER = [(i,j) for j in 2:N for i in 1:j-1]  # 15 positions
const PERMS = collect(permutations(1:N))           # All 720 permutations

"""Convert 15-bit mask to N×N adjacency matrix."""
function mask_to_adj(mask::UInt16)
    A = falses(N, N)
    p = 0
    @inbounds for (i,j) in UPPER
        p += 1
        if (mask >> (p-1)) & 0x1 == 0x1
            A[i,j] = true; A[j,i] = true
        end
    end
    return A
end

"""Permute adjacency matrix by π."""
function permute_adj(A::AbstractMatrix{Bool}, π::AbstractVector{Int})
    B = falses(N, N)
    @inbounds for i in 1:N, j in 1:N
        B[i,j] = A[π[i], π[j]]
    end
    return B
end

"""Check connectivity via BFS."""
function is_connected(A::AbstractMatrix{Bool})
    seen = falses(N); seen[1] = true
    q = [1]
    while !isempty(q)
        v = popfirst!(q)
        @inbounds for u in 1:N
            if A[v,u] && !seen[u]; seen[u]=true; push!(q,u); end
        end
    end
    return all(seen)
end

"""Canonical code for unlabeled graph (lex-min over all permutations)."""
function canon_code_graph(A::AbstractMatrix{Bool})
    best = nothing
    @inbounds for π in PERMS
        C = permute_adj(A, π)
        io = IOBuffer()
        for (i,j) in UPPER; write(io, C[i,j] ? '1' : '0'); end
        s = String(take!(io))
        if best === nothing || s < best; best = s; end
    end
    return best::String
end

"""Canonical code for rooted graph (graph + root position)."""
function canon_code_rooted(A::AbstractMatrix{Bool}, r::Int)
    best = nothing
    @inbounds for π in PERMS
        C = permute_adj(A, π)
        rp = π[r]
        io = IOBuffer()
        for (i,j) in UPPER; write(io, C[i,j] ? '1' : '0'); end
        s = String(take!(io)) * "|" * lpad(string(rp), 2, '0')
        if best === nothing || s < best; best = s; end
    end
    return best::String
end

"""Canonical code for two-layer multiplex with alignment σ."""
function canon_code_multiplex(A1::AbstractMatrix{Bool}, A2::AbstractMatrix{Bool}, 
                               r1::Int, r2::Int, σ::Vector{Int})
    A2s = permute_adj(A2, σ); r2s = σ[r2]
    best = nothing
    @inbounds for π in PERMS
        C1 = permute_adj(A1, π)
        C2 = permute_adj(A2s, π)
        rp1, rp2 = π[r1], π[r2s]
        io = IOBuffer()
        for (i,j) in UPPER; write(io, C1[i,j] ? '1' : '0'); end
        write(io, ';')
        for (i,j) in UPPER; write(io, C2[i,j] ? '1' : '0'); end
        write(io, ';'); write(io, lpad(string(rp1),2,'0')); 
        write(io, ','); write(io, lpad(string(rp2),2,'0'))
        s = String(take!(io))
        if best === nothing || s < best; best = s; end
    end
    return best::String
end

---
## 4. Build Edge/Strategy Sequences

In [ ]:
"""Build edge_seq and xi_seq for solver from adjacency matrices."""
function build_edge_xi(A1::AbstractMatrix{Bool}, A2::AbstractMatrix{Bool}, r1::Int, r2::Int)
    edges = Int[]
    for i in 1:N, j in i+1:N
        if A1[i,j]; append!(edges, (i,j,1,1)); append!(edges, (j,i,1,1)); end
        if A2[i,j]; append!(edges, (i,j,1,2)); append!(edges, (j,i,1,2)); end
    end
    edge_seq = reshape(edges, 4, length(edges)÷4)' |> x->Int64.(x)

    xi_seq = Array{Int64,2}(undef, 2N, 3)
    for i in 1:N
        xi_seq[i,:]   = [i, (i==r1) ? 1 : 0, 1]  # Layer 1
        xi_seq[N+i,:] = [i, (i==r2) ? 1 : 0, 2]  # Layer 2
    end
    return edge_seq, xi_seq
end

---
## 5. Enumeration Functions

In [ ]:
"""Enumerate all 112 non-isomorphic connected graphs on N=6."""
function all_connected_unlabeled_N6()
    reps = Dict{String, Matrix{Bool}}()
    for m in UInt16(0):UInt16(2^15-1)
        A = mask_to_adj(m)
        is_connected(A) || continue
        key = canon_code_graph(A)
        haskey(reps, key) || (reps[key] = A)
    end
    @info "Connected unlabeled graphs" count=length(reps)
    return collect(values(reps))
end

"""Generate rooted graph representatives (graph + mutant position)."""
function rooted_reps(graphs::Vector{Matrix{Bool}})
    roots = Dict{String, Tuple{Matrix{Bool}, Int}}()
    for A in graphs
        for r in 1:N
            key = canon_code_rooted(A, r)
            haskey(roots, key) || (roots[key] = (A, r))
        end
    end
    @info "Rooted configurations" count=length(roots)
    return collect(values(roots))
end

---
## 6. Main Sweep & CSV Output

In [ ]:
"""
    run_enumeration(outfile; compute_fn)

Enumerate all unique two-layer configurations and compute θ,φ.

# Arguments
- `outfile::String`: Output CSV path
- `compute_fn::Function`: θ,φ solver (default: bc_multilayer_DB)
"""
function run_enumeration(outfile::String; compute_fn::Function=bc_multilayer_DB)
    graphs = all_connected_unlabeled_N6()
    rooted = rooted_reps(graphs)
    
    seen = Set{String}()
    open(outfile, "w") do io
        println(io, "canon_id,theta1,theta2,theta3,phi01,phi20,phi21")
        count = 0
        total_pairs = length(rooted)^2 * factorial(N)
        
        for (A1, r1) in rooted, (A2, r2) in rooted
            for σ in PERMS
                key = canon_code_multiplex(A1, A2, r1, r2, σ)
                if !(key in seen)
                    push!(seen, key)
                    A2σ = permute_adj(A2, σ)
                    r2σ = σ[r2]
                    edge_seq, xi_seq = build_edge_xi(A1, A2σ, r1, r2σ)
                    θφ = compute_fn(edge_seq, xi_seq)
                    θ1, θ2, θ3 = θφ[2,1], θφ[3,1], θφ[4,1]
                    φ01, φ20, φ21 = θφ[2,2], θφ[3,2], θφ[4,2]
                    println(io, "$key,$θ1,$θ2,$θ3,$φ01,$φ20,$φ21")
                    count += 1
                    count % 10000 == 0 && @info "Progress" count
                end
            end
        end
        @info "Enumeration complete" unique_configs=count
    end
    return nothing
end

---
## 7. Execute Enumeration

In [ ]:
# Run the full enumeration (this may take several hours)
@time run_enumeration(OUTPUT_FILE; compute_fn=bc_multilayer_DB)
println("\nOutput written to: $OUTPUT_FILE")

---
## 8. Verify Output

In [ ]:
# Quick verification
if isfile(OUTPUT_FILE)
    lines = countlines(OUTPUT_FILE) - 1  # Exclude header
    println("Total unique configurations: $lines")
    
    # Preview first few rows
    open(OUTPUT_FILE) do f
        for (i, line) in enumerate(eachline(f))
            println(line)
            i >= 6 && break
        end
    end
end